# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import csv
import os
from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
df = pd.read_csv('cleanedData.csv',sep=',',encoding='utf-8')
df.head()
X = df['message'].astype(str)
Y = df.drop(columns=['message', 'original', 'id']).astype(str)
print(X.head())
print(Y.head())

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object
    genre related request offer aid_related medical_help medical_products  \
0  direct       1       0     0           0            0                0   
1  direct       1       0     0           1            0                0   
2  direct       1       0     0           0            0                0   
3  direct       1       1     0           1            0                1   
4  direct       1       0     0           0            0                0   

  search_and_rescue security military      ...      aid_centers  \
0                 0        0        0      ...                0   
1                 0        0        0      ...                0   
2                 0        0    

In [4]:
# load data from database
database_filepath = "cleanedDataBase1"
engine = create_engine(f'sqlite:///{os.path.abspath(database_filepath)}')
df = pd.read_sql_table('CleanedData',con=engine)  
X = df['message'].astype(str)
Y = df.drop(columns=['message', 'original', 'id', 'genre']).astype(str)

In [5]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [6]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize ,max_df=0.95, min_df=5)),  # Convert text to numerical features
    ('clf', MultiOutputClassifier(RandomForestClassifier()))  # Multi-output classification
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.95, max_features=None, min_df=5,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [10]:
X_train.head()

4149     Will there be any recruitment from the Nationa...
7012              What I do so that I can't get sore eyes 
5647     I'm a driver, I'm searching for a job since af...
24671    Bhachau: Blankets, Tarpaulin, GI Sheets, Bambo...
875      hi, i am a student. my house was destroyed. i'...
Name: message, dtype: object

In [11]:
Y_train.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
4149,1,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5647,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24671,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
875,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
Y_pred = pipeline.predict(X_test)

for i, col in enumerate(Y_test.columns):
    print(f"Category: {col}")
    print(classification_report(Y_test.iloc[:, i], Y_pred[:, i]))

Category: related
             precision    recall  f1-score   support

          0       0.61      0.36      0.45      1216
          1       0.82      0.93      0.87      4021
          2       0.54      0.37      0.43        41

avg / total       0.77      0.79      0.77      5278

Category: request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4353
          1       0.83      0.41      0.55       925

avg / total       0.88      0.88      0.86      5278

Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5255
          1       1.00      0.04      0.08        23

avg / total       1.00      1.00      0.99      5278

Category: aid_related
             precision    recall  f1-score   support

          0       0.71      0.87      0.79      3020
          1       0.76      0.53      0.62      2258

avg / total       0.73      0.73      0.72      5278

Category: med

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5239
          1       0.00      0.00      0.00        39

avg / total       0.99      0.99      0.99      5278

Category: hospitals
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      5225
          1       0.00      0.00      0.00        53

avg / total       0.98      0.99      0.98      5278

Category: shops
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5258
          1       0.00      0.00      0.00        20

avg / total       0.99      1.00      0.99      5278

Category: aid_centers
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      5219
          1       0.00      0.00      0.00        59

avg / total       0.98      0.99      0.98      5278

Category: other_infrastructure
             precision    recall  f1-score   suppor

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
parameters = {
    'clf__estimator__n_estimators': [50, 100],  # Number of trees in RandomForest
    'clf__estimator__max_depth': [10, None]  # Maximum depth of trees
}

grid_search = GridSearchCV(pipeline, parameters, cv=2, verbose=3, n_jobs=-1)

grid_search.fit(X_train, Y_train)

# Step 8: Display the best parameters
print("Best Parameters:", grid_search.best_params_)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50 ...
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=50 ...
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=100 ..
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=100 ..
[CV] clf__estimator__max_depth=None, clf__estimator__n_estimators=50 .
[CV] clf__estimator__max_depth=None, clf__estimator__n_estimators=50 .
[CV] clf__estimator__max_depth=None, clf__estimator__n_estimators=100 
[CV] clf__estimator__max_depth=None, clf__estimator__n_estimators=100 
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, score=0.1950919082812204, total= 3.2min
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=50, score=0.18684858821299982, total= 3.2min
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=100, score=0.18997536479060073, total= 4.7min
[CV]  clf__estimator__max_depth=10, clf__es

[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed:  5.7min remaining:  5.7min


[CV]  clf__estimator__max_depth=None, clf__estimator__n_estimators=50, score=0.25772219063862045, total= 8.6min
[CV]  clf__estimator__max_depth=None, clf__estimator__n_estimators=50, score=0.24265681258290694, total= 8.6min
[CV]  clf__estimator__max_depth=None, clf__estimator__n_estimators=100, score=0.2638809929884404, total=11.2min
[CV]  clf__estimator__max_depth=None, clf__estimator__n_estimators=100, score=0.24568883835512603, total=11.2min


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 12.0min finished


Best Parameters: {'clf__estimator__max_depth': None, 'clf__estimator__n_estimators': 100}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
Y_pred = grid_search.best_estimator_.predict(X_test)
# Step 10: Evaluate the model
for i, col in enumerate(Y_test.columns):
    print(f"Category: {col}")
    print(classification_report(Y_test.iloc[:, i], Y_pred[:, i]))

Category: related
             precision    recall  f1-score   support

          0       0.75      0.29      0.42      1216
          1       0.81      0.97      0.88      4021
          2       0.52      0.37      0.43        41

avg / total       0.80      0.81      0.77      5278

Category: request
             precision    recall  f1-score   support

          0       0.90      0.99      0.94      4353
          1       0.87      0.46      0.60       925

avg / total       0.89      0.89      0.88      5278

Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5255
          1       1.00      0.04      0.08        23

avg / total       1.00      1.00      0.99      5278

Category: aid_related
             precision    recall  f1-score   support

          0       0.76      0.87      0.81      3020
          1       0.79      0.63      0.70      2258

avg / total       0.77      0.77      0.77      5278

Category: med

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5239
          1       0.00      0.00      0.00        39

avg / total       0.99      0.99      0.99      5278

Category: hospitals
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      5225
          1       0.00      0.00      0.00        53

avg / total       0.98      0.99      0.98      5278

Category: shops
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5258
          1       0.00      0.00      0.00        20

avg / total       0.99      1.00      0.99      5278

Category: aid_centers
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      5219
          1       0.00      0.00      0.00        59

avg / total       0.98      0.99      0.98      5278

Category: other_infrastructure
             precision    recall  f1-score   suppor

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [31]:
print(X_test.head(2))
print(Y_test.head(2))
Y_pred = pipeline.predict(X_test.head(2))
print(Y_pred)

18018    Leaflets have been handed out, with pictures o...
428      Hello, we thank every organization, every inst...
Name: message, dtype: object
      related request offer aid_related medical_help medical_products  \
18018       1       0     0           0            0                0   
428         1       1     0           1            0                0   

      search_and_rescue security military child_alone      ...       \
18018                 0        0        0           0      ...        
428                   0        0        0           0      ...        

      aid_centers other_infrastructure weather_related floods storm fire  \
18018           0                    1               0      0     0    0   
428             0                    0               0      0     0    0   

      earthquake cold other_weather direct_report  
18018          0    0             0             0  
428            0    0             0             1  

[2 rows x 36 columns]
[['1' '0' '

### 9. Export your model as a pickle file

In [15]:
with open('best_model.pkl', 'wb') as f:
    pickle.dump(grid_search.best_estimator_, f)

print("Model saved as best_model.pkl")

Model saved as best_model.pkl


In [16]:
# Load with pickle
with open('best_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.